In [22]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tf # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

In [23]:
nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
services = ["Skill center", "Training center", "Start-up Center"]
part_of_the_day = ["matin", "soir "]
hours = [h for h in range(0,13)]
ms = [i for i in range(0,60)]
days = ('lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche')
free_time = [d+' ' + p+' à '+str(h)+':'+str(m) for d in days for p in part_of_the_day for h in hours for m in ms]
data = {"intents": [
    {"tag": "goodbye",
              "patterns": [ "bye", "later","Salut!","Au revoir","tchao","see ya", "adios", "cya","Bonne journée","Bonne soirée","À tout à l’heure","À plus tard","À bientôt","À demain", "À la prochaine", "à la prochaine fois","Bonne nuit", "Adieu"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!","Bye"]
             },
    {"tag": "greeting",
              "patterns": [ "Hi", "Hello", "Hey","Bonjour!", "Salut!","Bonsoir","Allô", "La forme?", "yo", "ça roule?"],
              "responses": ["Salut à toi!", "Comment vas tu?", "Salutations!", "Enchanté", "Hi :)"],
             },
    {
        "tag" : "schedule",
        "patterns" : ["Quels sont vos horaires?", "quels sont vos horaires de travail?","Quels sont les horaires de travail?"],
        "responses": ["de 09:00 AM à 18:00 PM"]
    },
    {
        "tag" : "services",
        "patterns" : ["Quels sont vos produits et services ?"],
        "responses": ["Tests hautement qualifiés pour déterminer le MEILLEUR, Certifications informatiques, Conseil, Coaching",
                      "Nous nous spécialisons dans l'utilisation de tests de compétences de scénarios modernes et éprouvés pour choisir parmi le TOP 1% des talents informatiques au Maroc.\nNous excellons dans la fourniture d'une formation de haute qualité pour être bien préparé pour passer votre certification informatique.\nNous mettons à votre disposition des consultants experts de haut niveau qui peuvent conduire votre entreprise vers le succès.\nNous boostons votre équipe pour qu'elle atteigne et dépasse ses objectifs en réalisant son plein potentiel en toute simplicité !"]
    },
    {
        "tag" : "about",
        "patterns" : ["qui êtes-vous?","Alors, qui êtes-vous?","Je souhaite bénéficier de vos services"],
        "responses": ["Architeo est un cabinet de consulting et conseil IT basé sur Rabat (Capitale du Maroc).\nNous nous spécialisons dans le portage salariale , la formations IT et le développement logiciel.\nNotre pool de talents opèrent dans differents branches et secteurs (Mobile, technique, digital, management, BI...)."]
    },
    {
        "tag" : "benefit",
        "patterns" : ["Comment bénéficier de vos services ?","puis-je bénéficier de vos services?","Je souhaite bénéficier de vos services"],
        "responses": ["de quel service vous souhaitez bénéficier?"]
    },
    {
        "tag" : "ask for appointment",
        "patterns" : ["Je veux prendre un rendez-vous","puis-je prendre rendez-vous avec un conseiller?","est-ce que je peux prendre un rendez-vous?","Je veux prendre rendez-vous pour avoir un conseil financier."],
        "responses": ["Avec plaisir quel est le meilleur moment pour vous?", "Avec plaisir quel est le moment qui vous convient?"]
    },
    {
        "tag" : "appointment time",
        "patterns" : [d+' ' + p+' à '+str(h)+':'+str(m) for d in days for p in part_of_the_day for h in hours for m in ms],
        "responses": ["d'accord c'est noté", "votre rendez-vous est confirmé", "nous ne sommes pas libres en ce moment, voici notre temps libre"]
    },

]}


In [25]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newWords.extend(ournewTkns)# extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [26]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)
       # print(bagOfwords)
    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])
# print(trainingData)
random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling
# print(trainingData)
x = num.array(list(trainingData[:, 0]))# first trainig phase
y = num.array(list(trainingData[:, 1]))# second training phase
# print(x,y)

In [36]:
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
# ourNewModel.add(Dense(857, input_shape=iShape, activation="relu"))
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax")) #35
# ourNewModel.add(Dense(oShape, activation = "sigmoid")) # 63
# below is a callable that returns the value to be used with no arguments
md = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Whilst training your Nural Network, you have the option of making the output verbose or simple.
ourNewModel.fit(x, y, epochs=200, verbose=1)
# By epochs, we mean the number of times you repeat a training set.

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 128)               109824    
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 64)                8256      
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_30 (Dense)            (None, 8)                 520       
                                                                 
Total params: 118,600
Trainable params: 118,600
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
343/343 [=====================

In [37]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [38]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, data)
    print(ourResult)

bonjour
Salut à toi!
qu est vos?
de quel service vous souhaitez bénéficier?
que etes vous
de quel service vous souhaitez bénéficier?
qui êtes-vous?
Architeo est un cabinet de consulting et conseil IT basé sur Rabat (Capitale du Maroc).
Nous nous spécialisons dans le portage salariale , la formations IT et le développement logiciel.
Notre pool de talents opèrent dans differents branches et secteurs (Mobile, technique, digital, management, BI...).
qui etes vous
Architeo est un cabinet de consulting et conseil IT basé sur Rabat (Capitale du Maroc).
Nous nous spécialisons dans le portage salariale , la formations IT et le développement logiciel.
Notre pool de talents opèrent dans differents branches et secteurs (Mobile, technique, digital, management, BI...).
qu'etes vous
de quel service vous souhaitez bénéficier?
vos services svp?
Architeo est un cabinet de consulting et conseil IT basé sur Rabat (Capitale du Maroc).
Nous nous spécialisons dans le portage salariale , la formations IT et l

KeyboardInterrupt: ignored